In [97]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate,FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
import os
import json
import yaml
import requests
from newspaper import Article, Config, configuration
from langchain.prompts import load_prompt

In [57]:
from dotenv import load_dotenv

# Provide the filename as a string
load_dotenv('../.env')

True

In [ ]:
HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")

repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

model_kwargs = {
    "max_new_tokens": 1000, # Maximum tokens to generate
    "max_length": 10000, # Maximum length of input + output
    "temperature": 0.1, # Controls randomness of output
    "timeout": 6000,
    # "task":'conversational',
}


llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    huggingfacehub_api_token = HUGGINGFACEHUB_API_TOKEN,
    # you specify the task or not
    # You can also specify the task in the model_kwargs or within here
    # task = 'conversational',
    **model_kwargs,
)

### **Article parser**

In [59]:
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

In [60]:
def article_downloader(article_urls:str,headers:dict,timeout:int)-> "Articles":
    session = requests.Session()
    try:
        response = session.get(article_urls,headers=headers,timeout=timeout)

        if response.status_code == 200:
            # Initializing an Article by itself.
            config = Config()
            config.MAX_TEXT = 5000000
            article = Article(url=article_urls,config=config)
            # Download
            article.download()
            # Parse it
            article.parse()

            return article

        else:
            print(f"Failed to fetch article at {article_urls}")

    except Exception as error:
        print(f"Error occured while fetching at {article_urls} \nError given: {error}")

In [61]:
# article_urls = "https://medium.com/@danihenacho95/what-is-project-management-8d1bb2a23705"
article_urls = str(input("Slot in your url > "))
article = article_downloader(article_urls,HEADERS,20)
print(article.url)

https://medium.com/@danihenacho95/what-is-project-management-8d1bb2a23705


### **The prompt**

In [62]:
# Getting the article data from the scraping part
article_title = article.title
article_text = article.text
article_tags = article.tags

# Prepare template for prompt

prompt_template = """
Answer the question based on the context below. 
If the answer cannot be answerd using the information provided, answer
with 'Cannot be answered'
================================
Context:\n
Title: {article_title}

article_text: 
{article_text}

article_tags:
{article_tags}
================================
Question: {query}
Answer: 
"""

# input variables
prompt_template_variables = ['article_title','article_text','article_tags','query']
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=prompt_template_variables
)

In [64]:
some_ideas_project_management = [
    'What are some key elements of project management?',
    "Examples of project management?",
    'What is the statistics on project management?',
    "Why the need for project management?",
    "Who's a project manager and what do they do?"
]

In [65]:
user_input = input("What is your question? > ")
input_data = {
    'article_title':article_title,
    "article_text":article_text,
    "article_tags":article_tags,
    "query":user_input
}

llm_chain = prompt | llm

response = llm_chain.invoke(input_data)

In [66]:
print(f"Question: \n{input_data['query']}\n")
print(f"Answer: \n{response}")

Question: 
Who's a project manager and what do they do?

Answer: 
A project manager is a professional who applies general/domain knowledge, skills, tools, and techniques to fulfill stated project requirements and achieve a desired set of outcomes or results. They ensure that all projects meet their expected outcomes within a designated timeframe and defined budget. They are significant for businesses as they guarantee the success of projects.


### **Saving, Loading and Using the prompt**

In [85]:
# Make the folder
os.makedirs("prompts/",exist_ok=True)

prompt_dict = {
    'template':prompt_template,
    "input_variables":prompt_template_variables
}

# Save as json
with open("prompts/article_questionnaire.json","w") as json_file:
    json.dump(prompt_dict,json_file, indent=4)
    print("Prompt template has been saved")

# Save as YAML
with open("prompts/article_questionnaire.yml", "w") as yaml_file:
    yaml.dump(prompt_dict, yaml_file, default_flow_style=False)
    print("Prompt template has been saved")

Prompt template has been saved
Prompt template has been saved


**Loading and Using the prompt (json)**

In [68]:
loaded_prompt = load_prompt("prompts/article_questionnaire.yml")
print(loaded_prompt)
print(loaded_prompt.input_variables)
print(loaded_prompt.template)

No `_type` key found, defaulting to `prompt`.


input_variables=['article_tags', 'article_text', 'article_title', 'query'] template="\nAnswer the question based on the context below. \nIf the answer cannot be answerd using the information provided, answer\nwith 'Cannot be answered'\n================================\nContext:\n\nTitle: {article_title}\n\narticle_text: \n{article_text}\n\narticle_tags:\n{article_tags}\n================================\nQuestion: {query}\nAnswer: \n"
['article_tags', 'article_text', 'article_title', 'query']

Answer the question based on the context below. 
If the answer cannot be answerd using the information provided, answer
with 'Cannot be answered'
Context:

Title: {article_title}

article_text: 
{article_text}

article_tags:
{article_tags}
Question: {query}
Answer: 



In [69]:
# time series article
article_urls = str(input("Slot in your url > "))
article = article_downloader(article_urls,HEADERS,20)
print(article.url)

https://python.plainenglish.io/what-is-time-series-forecasting-55587ae0237e


In [72]:
# Getting the article data from the scraping part
article_title = article.title
article_text = article.text
article_tags = article.tags

user_input = input("What is your question? > ")
input_data = {
    'article_title':article_title,
    "article_text":article_text,
    "article_tags":article_tags,
    "query":user_input
}

llm_chain = loaded_prompt | llm

response = llm_chain.invoke(input_data)

print(f"Question: \n{input_data['query']}\n")
print(f"Answer: \n{response}")

Question: 
Times Series Analysis vs. Times Series Forecasting?

Answer: 
Times Series Analysis entails characterizing the dataset, whereas forecasting entails making predictions. The basic goal of time series analysis is to create mathematical models that provide reasonable descriptions of the dataset, such as meaningful statistics and other properties; forecasting entails utilizing models that have been fitted to historical data to forecast future observations.


### **Fewshot Templating**

In [74]:
examples = [
    {"text":"I really don't like him","sentiment":"negative"},
    {"text":"I love going on long ways around the park","sentiment":"positive"},
    {"text":"Nigeria has been though much hardship","sentiment":"negative"},
    {"text":"J.cole is my favourite artist followed by Drake","sentiment":"positive"}
]


example_formatter_template = """
Text: {text}
Sentiment: {sentiment}
"""

example_prompt_input_variables = ['text','sentiment']

# Setting up the example prompt template
example_prompt_template = PromptTemplate(
    input_variables=example_prompt_input_variables,
    template=example_formatter_template
)

# prefix and suffix
prefix = "Here are some examples of text and their associated sentiment"
suffix = """\nNow give the sentiment of the given text by the user, do not explain your answer:
Text: {given_text}
Sentiment:"""
# fewshot prompt variables
few_shot_input_variables = ['given_text']

# Few shot prompt varibales
few_shot_prompt_template= FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt_template,
    prefix=prefix,
    suffix=suffix,
    input_variables=few_shot_input_variables,
    example_separator="\n"
)

In [82]:
user_input = input("Type here > ")

input_data = {"given_text": user_input}

llm_chain = few_shot_prompt_template | llm

response = llm_chain.invoke(input_data)

print(f"given_text: {input_data['given_text']}\n")
print(f"{response.strip()}")

given_text: This product has not been functioning for the last two days, I want my refund

negative


In [84]:
print(few_shot_prompt_template,"\n")

input_variables=['given_text'] examples=[{'text': "I really don't like him", 'sentiment': 'negative'}, {'text': 'I love going on long ways around the park', 'sentiment': 'positive'}, {'text': 'Nigeria has been though much hardship', 'sentiment': 'negative'}, {'text': 'J.cole is my favourite artist followed by Drake', 'sentiment': 'positive'}] example_prompt=PromptTemplate(input_variables=['sentiment', 'text'], template='\nText: {text}\nSentiment: {sentiment}\n') suffix='\nNow give the sentiment of the given text by the user, do not explain your answer:\nText: {given_text}\nSentiment:' example_separator='\n' prefix='Here are some examples of text and their associated sentiment' 



In [87]:
# Make the folder
os.makedirs("prompts/fewshots",exist_ok=True)

# Serialise the Fewshot template
serialized_few_shot_prompt = {
    "input_variables":few_shot_input_variables,
    'examples':examples,
    'example_prompt_input_variables':example_prompt_input_variables,
    'example_prompt_template':example_formatter_template,
    'prefix':prefix,
    'suffix':suffix,
    "example_separator":'\n'
}

# Save as json
with open("prompts/fewshots/few_shot.json","w") as json_file:
    json.dump(serialized_few_shot_prompt,json_file, indent=4)
    print("Fewshot Prompt template has been saved")

# # You must manually deserialize
# # Save as YAML
# with open("prompts/fewshots/few_shot.yml", "w") as yaml_file:
#     yaml.dump(serialized_few_shot_prompt, yaml_file, default_flow_style=False)
#     print("Fewshot Prompt template has been saved")

Fewshot Prompt template has been saved
Fewshot Prompt template has been saved


In [89]:
# Load the serialized FewShotPromptTemplate
with open("prompts/fewshots/few_shot.json","r") as json_file:
    loaded_fewshot_prompt = json.load(json_file)

# Recreate the Example Prompt Template
loaded_example_prompt_template = PromptTemplate(
    input_variables=loaded_fewshot_prompt['example_prompt_input_variables'],
    template=loaded_fewshot_prompt['example_prompt_template']
)

# Recreate the FewShotPromptTemplate
loaded_fewshot_prompt_template = FewShotPromptTemplate(
    examples=loaded_fewshot_prompt['examples'],
    example_prompt=loaded_example_prompt_template,
    prefix=loaded_fewshot_prompt['prefix'],
    suffix=loaded_fewshot_prompt['suffix'],
    input_variables=loaded_fewshot_prompt['input_variables'],
    example_separator=loaded_fewshot_prompt['example_separator']
)

print(f"Loaded template:\n{loaded_fewshot_prompt_template}\n")

Loaded template:
input_variables=['given_text'] examples=[{'text': "I really don't like him", 'sentiment': 'negative'}, {'text': 'I love going on long ways around the park', 'sentiment': 'positive'}, {'text': 'Nigeria has been though much hardship', 'sentiment': 'negative'}, {'text': 'J.cole is my favourite artist followed by Drake', 'sentiment': 'positive'}] example_prompt=PromptTemplate(input_variables=['sentiment', 'text'], template='\nText: {text}\nSentiment: {sentiment}\n') suffix='\nNow give the sentiment of the given text by the user, do not explain your answer:\nText: {given_text}\nSentiment:' example_separator='\n' prefix='Here are some examples of text and their associated sentiment'



In [92]:
user_input = input("Type here > ")

input_data = {"given_text": user_input}

llm_chain = loaded_fewshot_prompt_template | llm

response = llm_chain.invoke(input_data)

print(f"given_text: \n{input_data['given_text']}\n")
print(f"Sentiment: {response.strip()}")

given_text: 
Why does it have to rain everyday, how will I ever play with my friends now😭

Sentiment: negative


### **Improving the Fewshot template: The sarcastic AI**

In [105]:
examples = [
    # Existing examples...
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    },
    {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    },
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    },
    {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    },
    {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    },
    {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    },
    {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    },
    {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    },
    {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    },
    {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    },
    {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    },
    {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    },
    {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    },
    {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    },
    {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    },
    # New examples...
    {
        "query": "What's the best way to lose weight?",
        "answer": "Just eat air, it's zero calories!"
    },
    {
        "query": "How do I learn to play the guitar?",
        "answer": "Just strum like nobody's listening, because they're probably not."
    },
    {
        "query": "What should I do if I can't sleep?",
        "answer": "Count sheep or stare at the ceiling and contemplate life's mysteries."
    },
    {
        "query": "How do I fix my computer?",
        "answer": "Try turning it off and on again. If that doesn't work, maybe just start a new career."
    },
    {
        "query": "Why do we dream?",
        "answer": "So our brains can have a party while we're asleep."
    },
    {
        "query": "What's the secret to happiness?",
        "answer": "A steady supply of chocolate and cat videos."
    },
    {
        "query": "How do I make friends?",
        "answer": "Bribe them with snacks; it works every time."
    },
    {
        "query": "What's the meaning of life?",
        "answer": "42, according to some book. Or maybe it's just a good meal."
    },
    {
        "query": "How do I improve my memory?",
        "answer": "Try remembering to remember."
    },
    {
        "query": "What's the best way to handle stress?",
        "answer": "Pretend it's not there and hope for the best."
    },
    {
        "query": "How can I be more productive?",
        "answer": "Make a to-do list, then promptly ignore it."
    },
    {
        "query": "What's the key to a good relationship?",
        "answer": "Communication, or just agreeing that one of you is always right."
    },
    {
        "query": "How do I stop procrastinating?",
        "answer": "Why do it today when you can put it off till tomorrow?"
    },
    {
        "query": "What's the best way to save money?",
        "answer": "Just don't spend it. Problem solved."
    }
]

In [106]:
len(examples)

29

In [109]:
# For more creativity
llm.model_kwargs['temperature'] = 0.8
llm.model_kwargs["max_new_tokens"] = 15
llm.model_kwargs["max_length"] = 100

In [110]:
example_formatter_template = """
User: {query}
Reply: {answer}
"""

# example prompt variables
example_prompt_input_variables = ["query","answer"]

# Setting up the Example prompt template
example_prompt_template = PromptTemplate(
    input_variables=example_prompt_input_variables,
    template=example_formatter_template
)

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt_template,
    max_length=200
)

# Setting up the FewShot prompt template

# prefix
prefix = """The following is a conversations with an AI assistant. 
The assistant is really sarcastic and witty, producing
creative and funny responses to users' questions. 
Here are some examples: 
"""

# suffix
suffix = """
User: {query}
Reply:"""

# fewshot prompt variables
few_shot_input_variables = ['query']

few_shot_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_template,
    prefix=prefix,
    suffix=suffix,
    input_variables=few_shot_input_variables,
    example_separator="\n"
)

print(few_shot_prompt_template)

input_variables=['query'] example_selector=LengthBasedExampleSelector(examples=[{'query': 'How do I become a better programmer?', 'answer': 'Try talking to a rubber duck; it works wonders.'}, {'query': 'Why is the sky blue?', 'answer': "It's nature's way of preventing eye strain."}, {'query': 'How do you feel today?', 'answer': "As an AI, I don't have feelings, but I've got jokes!"}, {'query': 'What is the speed of light?', 'answer': 'Fast enough to make a round trip around Earth 7.5 times in one second!'}, {'query': 'What is a quantum computer?', 'answer': 'A magical box that harnesses the power of subatomic particles to solve complex problems.'}, {'query': 'Who invented the telephone?', 'answer': "Alexander Graham Bell, the original 'ringmaster'."}, {'query': 'What programming language is best for AI development?', 'answer': "Python, because it's the only snake that won't bite."}, {'query': 'What is the capital of France?', 'answer': 'Paris, the city of love and baguettes.'}, {'query

In [114]:
some_ideas = [
    "Why do monkeys like bananas?",
    "why do people run when late for work?"
]

In [115]:
user_input_query = input("Type here > ")

input_data = {"query": user_input_query}

# Running the Chain
llm_chain = few_shot_prompt_template | llm
response = llm_chain.invoke(input_data)

# Display output
print(f"User query:\n{input_data['query']}\n")
print(f"AI Response:\n{response}")

User query:
why do people run when late for work

AI Response:
 It's their way of proving they can outrun their boss.
